In [9]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\SummariesWithProcessAusGov'
outPath = r'P:\projects\RC10_ResultsSets'
summariesOutFolder = 'BaheerathanLanduseBudgets'
baseFolderEnd = 'BASE_RC10'
regContributorFileName = 'RegToOutlet.csv'
outputCSVEnd = 'LandusePollutantBudgets.csv'
baselineScenarioName = 'Baseline'
reportCardString = 'RC2019'

#pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'


##altRegLinker = 'Catchmt'
#standardRegName = 'RepReg'
##altRegColName = 'AUSGMCAS'
#origLinker = 'ModelElement'

#origLoadToStream = 'LoadToStream (kg)'
#origLoadToExport = 'LoadToRegExport (kg)'
annLoadToStream = "AnnLoadToStreamKG"
annLoadToExport = "AnnLoadToExportKG"

#regionIDs = ['MW']
regionIDs = ['BM','BU','CY','FI','MW','WT']
Constituents = ['Sediment - Fine', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']
#Processes = ['Gully', 'Hillslope surface soil', 'Streambank']
##FUs = ['Sugarcane', 'Grazing Open', 'Grazing Forested', 'Bananas']

scenarioToFolderDict = {baselineScenarioName:baseFolderEnd}


In [10]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [11]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

In [12]:
def getPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_' + reportCardString + '_' + baselineScenarioName + '_' + regContributorFileName
    fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [19]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    #Remove System Supply
    rawcontrib = rawcontrib.loc[~rawcontrib['FU'].isin(['System Supply'])]
    
    filteredFlow = rawcontrib.loc[rawcontrib['Constituent'] == 'Flow']
    
    flowTotal = pd.DataFrame(filteredFlow.groupby(['Rep_Region','FU']).agg({annLoadToStream:'sum'})).reset_index()
    flowTotal.rename(columns={annLoadToStream:'AnnualRunoffLitres'}, inplace=True)
    
    filteredConstit = rawcontrib.loc[rawcontrib['Constituent'].isin(Constituents)]
    #processFiltered = filteredConstit.loc[filteredConstit['Process'].isin(Processes)]
    processFiltered = filteredConstit
    
    consForArea = Constituents[0]#Probably sediemnt - fine, doesn't matter
    #then lets use Gully...
    areaFiltered = processFiltered.loc[processFiltered['Constituent'] == consForArea]
    areaFiltered = areaFiltered.loc[areaFiltered['Process'] == 'Gully']
    areaTotals = pd.DataFrame(areaFiltered.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    regNonProcessRSDR = pd.DataFrame(processFiltered.groupby(['Rep_Region','Constituent']).agg({annLoadToStream:'sum', annLoadToExport:'sum'})).reset_index()
    regNonProcessRSDR['RegRSDR'] = regNonProcessRSDR[annLoadToExport] / regNonProcessRSDR[annLoadToStream]
    #where 1 is the axis number (0 for rows and 1 for columns.)
    regNonProcessRSDR = regNonProcessRSDR.drop(annLoadToStream, 1)
    regNonProcessRSDR.rename(columns={annLoadToExport:'RegTotalAnnLoadToExportKG'}, inplace=True)
    #regNonProcessRSDR = regNonProcessRSDR.drop(annLoadToExport, 1)  
    
    regProcessTotals = pd.DataFrame(processFiltered.groupby(['Rep_Region','Constituent','FU']).agg({annLoadToStream:'sum'})).reset_index()
    regProcessTotals = pd.merge(regProcessTotals, flowTotal, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    regProcessTotals = pd.merge(regProcessTotals, regNonProcessRSDR, how='left', left_on=['Rep_Region','Constituent'], right_on = ['Rep_Region','Constituent'])
    regProcessTotals['AdjAnnLoadToExportKG'] = regProcessTotals[annLoadToStream] * regProcessTotals['RegRSDR']
    regProcessTotals['PercExport'] = regProcessTotals['AdjAnnLoadToExportKG'] / regProcessTotals['RegTotalAnnLoadToExportKG'] * 100
    regProcessTotals = pd.merge(regProcessTotals, areaTotals, how='outer', on=['Rep_Region','FU'])
    

    regProcessTotals.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [20]:
#List out the processes

for theReg in regionIDs:
    
    for scenario in scenarioToFolderDict.keys():
        
        print("Doing: " + theReg + " " + scenario)
        produceSummaryFromRegContrib(theReg, scenario)



print("Finished first summary")

Doing: BM Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\BM_RC2019_Baseline_RegToOutlet.csv Scenario: Baseline


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Saved P:\projects\RC10_ResultsSets\BaheerathanLanduseBudgets\BM_RC2019_Baseline_LandusePollutantBudgets.csv
Doing: BU Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\BU_RC2019_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\BaheerathanLanduseBudgets\BU_RC2019_Baseline_LandusePollutantBudgets.csv
Doing: CY Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\CY_RC2019_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\BaheerathanLanduseBudgets\CY_RC2019_Baseline_LandusePollutantBudgets.csv
Doing: FI Baseline
Processing \\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\FI_RC2019_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC10_ResultsSets\BaheerathanLanduseBudgets\FI_RC2019_Baseline_LandusePollutantBudgets.csv
Doing: MW Baseline
Processing \